#Movie Recommendation System


##Objective
The primary goal of this project is to develop a movie recommendation system based on content-based filtering. The system leverages natural language processing and machine learning techniques to suggest movies similar to a user's input, considering movie features such as genre, keywords, tagline, cast, and director.

##Data Source
Dataset from Github: YBIFoundation\Dataset\MoviesRecommendation.csv

##Import Libraries

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [71]:
from sklearn.metrics.pairwise import cosine_similarity

In [72]:
import difflib

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

##Import Data

In [57]:
df = pd.read_csv('https://github.com/YBIFoundation/Dataset/raw/main/Movies%20Recommendation.csv')

In [58]:
df.head()

,Movie_ID,Movie_Title,Movie_Genre,Movie_Language,Movie_Budget,Movie_Popularity,Movie_Release_Date,Movie_Revenue,Movie_Runtime,Movie_Vote,...,Movie_Homepage,Movie_Keywords,Movie_Overview,Movie_Production_House,Movie_Production_Country,Movie_Spoken_Language,Movie_Tagline,Movie_Cast,Movie_Crew,Movie_Director
0,1,Four Rooms,Crime Comedy,en,4000000,22.876230,09-12-1995,4300000,98.0,6.5,...,NaN,hotel new year's eve witch bet hotel room,It's Ted the Bellhop's first night on the job....,"[{""name"": ""Miramax Films"", ""id"": 14}, {""name"":...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",Twelve outrageous guests. Four scandalous requ...,Tim Roth Antonio Banderas Jennifer Beals Madon...,"[{'name': 'Allison Anders', 'gender': 1, 'depa...",Allison Anders
1,2,Star Wars,Adventure Action Science Fiction,en,11000000,126.393695,25-05-1977,775398007,121.0,8.1,...,http://www.starwars.com/films/star-wars-episod...,android galaxy hermit death star lightsaber,Princess Leia is captured and held hostage by ...,"[{""name"": ""Lucasfilm"", ""id"": 1}, {""name"": ""Twe...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","A long time ago in a galaxy far, far away...",Mark Hamill Harrison Ford Carrie Fisher Peter ...,"[{'name': 'George Lucas', 'gender': 2, 'depart...",George Lucas
2,3,Finding Nemo,Animation Family,en,94000000,85.688789,30-05-2003,940335536,100.0,7.6,...,http://movies.disney.com/finding-nemo,father son relationship harbor underwater fish...,"Nemo, an adventurous young clownfish, is unexp...","[{""name"": ""Pixar Animation Studios"", ""id"": 3}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","There are 3.7 trillion fish in the ocean, they...",Albert Brooks Ellen DeGeneres Alexander Gould ...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton
3,4,Forrest Gump,Comedy Drama Romance,en,55000000,138.133331,06-07-1994,677945399,142.0,8.2,...,NaN,vietnam veteran hippie mentally disabled runni...,A man with a low IQ has accomplished great thi...,"[{""name"": ""Paramount Pictures"", ""id"": 4}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]","The world will never be the same, once you've ...",Tom Hanks Robin Wright Gary Sinise Mykelti Wil...,"[{'name': 'Alan Silvestri', 'gender': 2, 'depa...",Robert Zemeckis
4,5,American Beauty,Drama,en,15000000,80.878605,15-09-1999,356296601,122.0,7.9,...,http://www.dreamworks.com/ab/,male nudity female nudity adultery midlife cri...,"Lester Burnham, a depressed suburban father in...","[{""name"": ""DreamWorks SKG"", ""id"": 27}, {""name""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...","[{""iso_639_1"": ""en"", ""name"": ""English""}]",Look closer.,Kevin Spacey Annette Bening Thora Birch Wes Be...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes


##Describing Data

In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4760 entries, 0 to 4759
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Movie_ID                  4760 non-null   int64  
 1   Movie_Title               4760 non-null   object 
 2   Movie_Genre               4760 non-null   object 
 3   Movie_Language            4760 non-null   object 
 4   Movie_Budget              4760 non-null   int64  
 5   Movie_Popularity          4760 non-null   float64
 6   Movie_Release_Date        4760 non-null   object 
 7   Movie_Revenue             4760 non-null   int64  
 8   Movie_Runtime             4758 non-null   float64
 9   Movie_Vote                4760 non-null   float64
 10  Movie_Vote_Count          4760 non-null   int64  
 11  Movie_Homepage            1699 non-null   object 
 12  Movie_Keywords            4373 non-null   object 
 13  Movie_Overview            4757 non-null   object 
 14  Movie_Pr

In [60]:
df.shape

(4760, 21)

In [61]:
df.columns

Index(['Movie_ID', 'Movie_Title', 'Movie_Genre', 'Movie_Language',
       'Movie_Budget', 'Movie_Popularity', 'Movie_Release_Date',
       'Movie_Revenue', 'Movie_Runtime', 'Movie_Vote', 'Movie_Vote_Count',
       'Movie_Homepage', 'Movie_Keywords', 'Movie_Overview',
       'Movie_Production_House', 'Movie_Production_Country',
       'Movie_Spoken_Language', 'Movie_Tagline', 'Movie_Cast', 'Movie_Crew',
       'Movie_Director'],
      dtype='object')

##Data Preprocessing

In [62]:
df_features = df[['Movie_Genre','Movie_Keywords','Movie_Tagline','Movie_Cast','Movie_Director']].fillna('')

In [63]:
df_features.shape

(4760, 5)

In [64]:
df_features

,Movie_Genre,Movie_Keywords,Movie_Tagline,Movie_Cast,Movie_Director
0,Crime Comedy,hotel new year's eve witch bet hotel room,Twelve outrageous guests. Four scandalous requ...,Tim Roth Antonio Banderas Jennifer Beals Madon...,Allison Anders
1,Adventure Action Science Fiction,android galaxy hermit death star lightsaber,"A long time ago in a galaxy far, far away...",Mark Hamill Harrison Ford Carrie Fisher Peter ...,George Lucas
2,Animation Family,father son relationship harbor underwater fish...,"There are 3.7 trillion fish in the ocean, they...",Albert Brooks Ellen DeGeneres Alexander Gould ...,Andrew Stanton
3,Comedy Drama Romance,vietnam veteran hippie mentally disabled runni...,"The world will never be the same, once you've ...",Tom Hanks Robin Wright Gary Sinise Mykelti Wil...,Robert Zemeckis
4,Drama,male nudity female nudity adultery midlife cri...,Look closer.,Kevin Spacey Annette Bening Thora Birch Wes Be...,Sam Mendes
...,...,...,...,...,...
4755,Horror,,The hot spot where Satan's waitin'.,Lisa Hart Carroll Michael Des Barres Paul Drak...,Pece Dingo
4756,Comedy Family Drama,,It’s better to stand out than to fit in.,Roni Akurati Brighton Sharbino Jason Lee Anjul...,Frank Lotito
4757,Thriller Drama,christian film sex trafficking,She never knew it could happen to her...,Nicole Smolen Kim Baldwin Ariana Stephens Brys...,Jaco Booyens
4758,Family,,,,


In [65]:
x = df_features['Movie_Genre']+' '+df_features['Movie_Keywords']+' '+df_features['Movie_Tagline']+' '+df_features['Movie_Director']

In [66]:
x

0       Crime Comedy hotel new year's eve witch bet ho...
1       Adventure Action Science Fiction android galax...
2       Animation Family father son relationship harbo...
3       Comedy Drama Romance vietnam veteran hippie me...
4       Drama male nudity female nudity adultery midli...
                              ...                        
4755    Horror  The hot spot where Satan's waitin'. Pe...
4756    Comedy Family Drama  It’s better to stand out ...
4757    Thriller Drama christian film sex trafficking ...
4758                                            Family   
4759    Documentary music actors legendary perfomer cl...
Length: 4760, dtype: object

In [67]:
x.shape

(4760,)

In [68]:
x = tfidf.fit_transform(x)

In [69]:
x.shape

(4760, 9666)

In [70]:
print(x)

  (0, 376)	0.18272810724070418
  (0, 316)	0.19632202064896634
  (0, 5024)	0.09373465698968353
  (0, 6094)	0.06761788284030082
  (0, 9426)	0.19090152648212289
  (0, 3256)	0.09303304831704608
  (0, 9401)	0.11559559866313328
  (0, 4521)	0.1443365785980048
  (0, 8543)	0.1065109136318404
  (0, 6131)	0.0786972588733094
  (0, 2203)	0.1375276201579133
  (0, 3175)	0.12385671298754733
  (0, 4023)	0.21434670508652215
  (0, 4253)	0.15944969524116445
  (0, 866)	0.21315961343110615
  (0, 5106)	0.16963506748686155
  (0, 6133)	0.08935032151608546
  (0, 374)	0.08264227113134892
  (0, 7062)	0.21315961343110615
  (0, 7448)	0.21315961343110615
  (0, 3296)	0.15847233257383517
  (0, 3724)	0.20331025305114117
  (0, 6209)	0.20331025305114117
  (0, 8864)	0.19090152648212289
  (0, 7244)	0.15847233257383517
  :	:
  (4757, 1975)	0.26800244457573114
  (4757, 3821)	0.29309239322653075
  (4757, 7673)	0.22148093071406288
  (4757, 7622)	0.20123579506754563
  (4757, 3145)	0.168092031177192
  (4757, 4761)	0.316532199084

##Modeling

In [73]:
Similarity_Score = cosine_similarity(x)

In [74]:
Similarity_Score

array([[1.        , 0.01723123, 0.04583093, ..., 0.        , 0.        ,
        0.        ],
       [0.01723123, 1.        , 0.01127435, ..., 0.        , 0.        ,
        0.        ],
       [0.04583093, 0.01127435, 1.        , ..., 0.        , 0.09498713,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.09498713, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [75]:
Similarity_Score.shape

(4760, 4760)

##Prediction based on input by user

In [94]:
Favourite_Movie_Name = input('Enter your current favourite movie name : ')

Enter your current favourite movie name : star wars


In [95]:
All_Movies_Title_List = df['Movie_Title'].tolist()

In [96]:
Movie_Recommendation = difflib.get_close_matches(Favourite_Movie_Name, All_Movies_Title_List)

In [97]:
print(Movie_Recommendation)

['Star Wars']


In [98]:
Close_Match = Movie_Recommendation[0]

In [99]:
print(Close_Match)

Star Wars


In [100]:
Index_of_Close_Match_Movie = df[df.Movie_Title == Close_Match]['Movie_ID'].values[0]

In [101]:
print(Index_of_Close_Match_Movie)

2


In [102]:
Recommendation_Score = list(enumerate(Similarity_Score[Index_of_Close_Match_Movie]))

In [103]:
print(Recommendation_Score)

[(0, 0.045830934084954554), (1, 0.011274348352070819), (2, 1.0), (3, 0.014676946430384837), (4, 0.0), (5, 0.0), (6, 0.026803498384208608), (7, 0.03273227179437856), (8, 0.0), (9, 0.005426303413476344), (10, 0.027728053042969467), (11, 0.005898071062297188), (12, 0.006754134135618732), (13, 0.0), (14, 0.11184266910146456), (15, 0.0), (16, 0.0), (17, 0.0), (18, 0.005634384320678318), (19, 0.0), (20, 0.0), (21, 0.0), (22, 0.04538226208254131), (23, 0.0), (24, 0.05787597149330078), (25, 0.01803388067086491), (26, 0.01794080589996335), (27, 0.05493450369867333), (28, 0.005975826193273931), (29, 0.0240358153017253), (30, 0.026016416899615067), (31, 0.0), (32, 0.016737057634357122), (33, 0.023595740567800744), (34, 0.0), (35, 0.01577905555131449), (36, 0.0), (37, 0.01616125019515528), (38, 0.007384202708402788), (39, 0.011783478224979146), (40, 0.0), (41, 0.0), (42, 0.0), (43, 0.03376772270363138), (44, 0.025552888350109375), (45, 0.015082294398915019), (46, 0.0), (47, 0.0), (48, 0.0070318580

In [104]:
len(Recommendation_Score)

4760

In [105]:
Sorted_Similar_Movies = sorted(Recommendation_Score, key=lambda x:x[1], reverse = True)

In [106]:
print(Sorted_Similar_Movies)

[(2, 1.0), (1714, 0.26320444597156134), (1758, 0.22227134619735622), (3256, 0.19043466477557597), (247, 0.1839234097207056), (2135, 0.17659918813061784), (709, 0.1742194068316649), (2433, 0.17105071798087565), (1473, 0.1701699335044824), (3181, 0.1630843277963322), (3390, 0.1599763905556326), (4182, 0.15104891653713715), (1276, 0.1499249678954277), (2098, 0.1498809308847658), (3467, 0.1480144082143679), (1327, 0.14577452134913163), (3144, 0.1454988375756734), (2897, 0.14378343540226363), (1182, 0.1437696247558918), (1319, 0.1432365040945076), (2322, 0.1394475689311069), (2591, 0.13825576420098137), (3471, 0.13688451996803314), (106, 0.13419807722886093), (293, 0.13398728672063773), (2335, 0.13370597893759084), (1477, 0.13320275621634062), (3632, 0.13010451202586867), (1886, 0.1276637336855593), (2264, 0.12716913713667571), (3852, 0.12558814706017432), (3870, 0.12527872044452712), (1072, 0.12462794252217854), (1524, 0.1243298448522987), (1737, 0.12388300574121527), (3871, 0.120181042268

##Output

In [107]:
print('Top Movies Recommended for You : \n')
i = 1

for movie in Sorted_Similar_Movies:
   index = movie[0]
   title_from_index = df[df.index==index]['Movie_Title'].values[0]
   if(i<31):
      print(f"{i}. {title_from_index}")
      i+=1

Top Movies Recommended for You : 

1. Finding Nemo
2. Shark Tale
3. WALL·E
4. Meet the Deedles
5. Big Fish
6. Ponyo
7. Mallrats
8. Jonah: A VeggieTales Movie
9. Happy Feet
10. Megamind
11. When Did You Last See Your Father?
12. Treading Water
13. El Mariachi
14. White Chicks
15. John Carter
16. A Bug's Life
17. Oceans
18. Shooting Fish
19. Dear Frankie
20. He Got Game
21. Zoom
22. Because of Winn-Dixie
23. The Reef
24. The Outsiders
25. Central Station
26. Aquamarine
27. My Favorite Martian
28. Dark Shadows
29. Tora! Tora! Tora!
30. Bottle Rocket


##Explanation
Input: User provides the name of their current favorite movie.  
Processing: Find the closest match in the dataset using the difflib.get_close_matches function.  
Recommendation: Use cosine similarity to recommend movies similar to the user's favorite.  
The code prints the top suggested movies based on similarity scores.This project aims to enhance user experience by providing personalized movie recommendations using content-based filtering. The system considers various movie features to suggest similar movies based on user preferences.